In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import time
from bs4 import BeautifulSoup
import json
from pathlib import Path
import requests
from tqdm import tqdm

In [ ]:
course_url = []
while True:
        url = input('Enter url: \n')
        if url == 'c':
            break
        else:
            course_url.append(url)
            
with open('course_url.json', 'w') as file:
    json.dump(course_url, file)

In [ ]:
options = webdriver.ChromeOptions()
#options.add_argument("--headless")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

driver = webdriver.Chrome(options=options, executable_path=r"\path\to\chromedriver.exe")

driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
  "source": """
    Object.defineProperty(navigator, 'webdriver', {
      get: () => undefined
    })
  """
})
driver.execute_cdp_cmd("Network.enable", {})
driver.execute_cdp_cmd("Network.setExtraHTTPHeaders", {"headers": {"User-Agent": "browser1"}})


wait = WebDriverWait(driver, 10)
driver.switch_to.window(driver.current_window_handle)
loginurl = "https://courses.edx.org/login"
driver.get(loginurl)
login_element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.action.action-primary.action-update.js-login.login-button')))
username_element = driver.find_element_by_name('email')
password_element = driver.find_element_by_name('password')

username = 'username'
password = 'password'

username_element.send_keys(username)
password_element.send_keys(password)
login_element.click()

time.sleep(6)

with open('course_url.json', 'r') as file:
    course_url = json.load(file)

for url in course_url:
    driver.get(url)
    driver.switch_to.window(driver.current_window_handle)
    video_list = []
    while True:
        try:
            next_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'button.sequence-nav-button.button-next')))
            next_button.click()
            time.sleep(2)
            video_present = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.xblock.xblock-student_view.xblock-student_view-video.xmodule_display.xmodule_VideoBlock.xblock-initialized')))
            
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            video = soup.find_all(class_ = "xblock xblock-student_view xblock-student_view-video xmodule_display xmodule_VideoBlock xblock-initialized")
            matchobj = re.search('\[\"https(.*)mp4', video[0].div['data-metadata'])
            video_link = re.sub('[["]', '', matchobj.group())

            chapter = soup.find_all(class_ = 'nav-item nav-item-chapter')
            folder_level_1 = re.sub('[:,\n]', '', chapter[0].text)

            section = soup.find_all(class_ = 'nav-item nav-item-section')
            folder_level_2 = re.sub('[:,\n]', '', section[0].text)

            title = soup.find_all(class_ = 'nav-item nav-item-sequence')
            video_title = re.sub('[:,\n]', '', title[0].text)

            course = soup.find_all(class_ = 'course-name')
            course_title = re.sub('[:,\n]', '', course[0].text)

            video_info = {'video_link': video_link,
                          'video_title': video_title,
                          'root_folder': course_title,
                          'folder_level_1': folder_level_1, 
                          'folder_level_2': folder_level_2}
            video_list.append(video_info)
        except (StaleElementReferenceException, TimeoutException):
            continue
        except Exception as e:
            print(e)
            break
    
    with open('video_list_' + str(course_url.index(url)) + '.json', 'w') as file:
        json.dump(video_list, file) 
        
driver.quit()

In [ ]:
with open('video_list_{}.json'.format(0), 'r') as file:
    video_list = json.load(file)

In [ ]:
root = Path(r'path\to\downloads')

for i in range(1, 7):
    with open('video_list_{}.json'.format(i), 'r') as file:
        video_list = json.load(file)

    for video in video_list:
        video_title = re.sub('[:,?\/*<>"]', '', video['video_title'])
        root_folder = re.sub('[:,?\/*<>"]', '', video['root_folder'])
        folder_level_1 = re.sub('[:,?\/*<>"]', '', video['folder_level_1'])
        folder_level_2 = re.sub('[:,?\/*<>"]', '', video['folder_level_2'])
        video_path = root.joinpath(root_folder, folder_level_1, folder_level_2)
        video_path.mkdir(parents=True, exist_ok=True)
        video_link = re.sub('https(.*)m3u8, ', '', video['video_link'])
        r = requests.get(video_link, stream = True)
        with open(video_path.joinpath(str(video_list.index(video)) + '. ' + video_title + '.mp4'), 'wb') as video:
            for chunk in tqdm(r.iter_content(chunk_size = 1000)):
                video.write(chunk)
    print('Download complete')
            
print('All downloads complete!')